# Ziroom 北京市租房信息（包含房价）采集（python）

## 思路分析

- 一级网页为大循环，主要操作二级网页
    - 一级网页：包含多个详细房源信息的网页，用于获取详细房源信息，翻页
    - 二级网页：详细房源信息，用于获取具体的信息
- 由于网络情况很可能出现部分页面虽然是包含目标信息的但是没有成功获取的情况，所以要储存加载失败的网页链接，以便进行进一步的补充获取
- 房价信息较难获取，在获取信息的时候暂时先只保存背景图片的网址以及各位数字在图片中的位置
- 使用 ```BeautifulSoup``` 包进行网络信息获取
- 注意设置异常情况（如网页没有加载成功，标签不存在）的处理，以保证爬虫的稳定

## 导入使用的包

In [1]:
from bs4 import BeautifulSoup
import re
from urllib.request import urlopen
import pandas as pd
from urllib.error import HTTPError
from PIL import Image
import pytesseract
from io import BytesIO
import requests
import copy

## 读取原始数据

需要函数：
- 找到网页中所有的具体房源链接
- 找到具体房源链接中所有有用的信息

In [3]:
# 筛选页面中所有可以链接到具体房间页面的链接
# 传入的变量为一个链接
# 返回所有符合要求链接的列表（无重复）
def getLinks(url):
    # 生成该网页的 BeautifulSoup 对象
    for _ in range(20):
        html = urlopen(url)
        bso = BeautifulSoup(html.read())
        if len(bso) > 0:
            break
    # 20次尝试以后还是没有加载成功
    if len(bso) == 0:
        print("该 BeautifulSoup 对象为空，返回 []")
        return []
    
    links = [] # 用于储存最终的结果
    links_all = bso.findAll("a", {"href":re.compile("//www.ziroom.com/x/.*html")})
    for link in links_all:
        # 只有显示出现的链接才是我们感兴趣的量
        if 'href' in link.attrs:
            links.append('http:' + link.attrs['href'])
    if len(links) == 0:
        print("出错！该 BeautifulSoup 对象虽然不为空，但是没有出现符合条件的链接，原本的网页长这个样子")
        print(bso)
        print("该网页的网址为：", url)
    print(url, "中所有具体房源链接返回成功！一共有（个）：", len(set(links)))
    return list(set(links))

In [78]:
# 传入一个具体房源页面的网址
# 以数据框的形式输出该页面上所有需要获取的信息
'''
获取信息包括：
    名称：name
    房屋面积：area
    房屋朝向：toward
    房屋配置（几室几厅）：config
    交通距离（尚不明确是否均为距离地铁站的距离）：distance
    房屋楼层/总楼层：floor/total
    是否有电梯：have_elevator
    房屋建成年份：built_year
    如何供暖：heat
    绿化程度（百分比）：green
    价格对应的背景图片链接：img_url
    价格的源代码信息：price_raw
    价格单位（个别房间是按天计价）：price_unit
    网页链接（方便后期检查）：url
'''
def getInf(url, successLinks_in, failLinks_in):
    # 生成该网页的 BeautifulSoup 对象
    # 20次之内如果找不到该网页就放弃
    successLinks = successLinks_in
    failLinks = failLinks_in
    for i in range(20):
#         if i > 18:
#             print("尝试打开这个链接很多次")
        html = urlopen(url)
        bso = BeautifulSoup(html.read())
        if len(bso) > 0:
            break
    #出现这种问题应该回收这个链接
    if len(bso) == 0:
        failLinks.append(url)
#         print("该 BeautifulSoup 对象完全就是空的，更找不到具体明细了，返回 None")
#         print("出错网页：", url)
        return None
    
    try:
        sideInf = bso.body.section.aside
        homeInf = sideInf.find("div", {"class": "Z_home_info"})
        inf = {}
        inf['name'] = sideInf.h1.get_text()
        inf['area'], inf['toward'], inf['config'] = [dd.get_text() for dd in homeInf.findAll("dd")]
        inf['distance'], inf['floor/total'], inf['have_elevator'], inf['built_year'], inf['heat'], inf['green'] = [s.get_text() for s in homeInf.findAll("span", {"class": "va"})]
        
        # 存储价格有关的原始信息，是列表的形式
        priceInf = sideInf.find("div", {"class": "Z_price"})
        numsInf = priceInf.findAll("i", {"class": "num"})
        unitInf = priceInf.findAll("span")[-1]
        
        imgUrl = 'https:' + re.findall("//[.|0-9|a-z|A-Z|/]*", str(numsInf[0]))[0]
        priceIdxRaw = []
        for numInf in numsInf:
            pos = re.findall("[0-9|.]*px", str(numInf))[0][:-2]
            priceIdxRaw.append(str(pos))
        inf['img_url'] = imgUrl
        inf['price_raw'] = ','.join(priceIdxRaw)
        inf['price_unit'] = unitInf.get_text()
        inf['url'] = url
#         print(url, "上的数据项已采集完全！成功返回！")
        
        successLinks.append(url)
        return pd.DataFrame(inf,index = [0])
    
    except AttributeError as e:
        # 只要有一个值没有没采集就整个返回为空
#         print(url, "上的数据项没有采集完全，返回 None")
        return None

In [75]:
# 返回传入页面链接上所有感兴趣的变量
def getPagesInf(links, successLinks_in, failLinks_in):
    successLinks = successLinks_in
    failLinks = failLinks_in
    noneCounter = 0
    inf_df = pd.DataFrame()
    for i, link in enumerate(links):
#         最终运行的输出提示
        if i%100 == 0:
            print("已经运行到第", i, "个链接，成功采集条数：", len(successLinks), "失败采集条数：", len(failLinks))
        try:
            pageSuccessLinks = []
            pageFailLinks = []
            inf = getInf(link, pageSuccessLinks, pageFailLinks)
            successLinks.extend(pageSuccessLinks)
            failLinks.extend(pageFailLinks)
            inf_df = inf_df.append(inf)
            if type(inf_df) == None:
                noneCounter += 1
            if i > 100 and noneCounter > i/2:
                print("这次循环出现过多采集失败，未采集到的链接归入失败列表")
                failLinks.extend(links[i:])
                return inf_df
        except HTTPError as e:
            print(link, '打不开')
            failLinks.append(link)
    return inf_df

In [21]:
# 注意到一共有50个页面，可以直接设置死的页数循环
# 待修改为更加灵活的循环
pagesLinks = []
for i in range(1, 51):
    url = ('https://www.ziroom.com/z/z0-p{}/?qwd=%E5%8C%97%E4%BA%AC'.format(i))
    thisPageLinks = getLinks(url)
    pagesLinks.extend(thisPageLinks)

https://www.ziroom.com/z/z0-p1/?qwd=%E5%8C%97%E4%BA%AC 中所有具体房源链接返回成功！一共有（个）： 59
https://www.ziroom.com/z/z0-p2/?qwd=%E5%8C%97%E4%BA%AC 中所有具体房源链接返回成功！一共有（个）： 59
https://www.ziroom.com/z/z0-p3/?qwd=%E5%8C%97%E4%BA%AC 中所有具体房源链接返回成功！一共有（个）： 59
https://www.ziroom.com/z/z0-p4/?qwd=%E5%8C%97%E4%BA%AC 中所有具体房源链接返回成功！一共有（个）： 60
https://www.ziroom.com/z/z0-p5/?qwd=%E5%8C%97%E4%BA%AC 中所有具体房源链接返回成功！一共有（个）： 60
https://www.ziroom.com/z/z0-p6/?qwd=%E5%8C%97%E4%BA%AC 中所有具体房源链接返回成功！一共有（个）： 59
https://www.ziroom.com/z/z0-p7/?qwd=%E5%8C%97%E4%BA%AC 中所有具体房源链接返回成功！一共有（个）： 59
https://www.ziroom.com/z/z0-p8/?qwd=%E5%8C%97%E4%BA%AC 中所有具体房源链接返回成功！一共有（个）： 58
https://www.ziroom.com/z/z0-p9/?qwd=%E5%8C%97%E4%BA%AC 中所有具体房源链接返回成功！一共有（个）： 58
https://www.ziroom.com/z/z0-p10/?qwd=%E5%8C%97%E4%BA%AC 中所有具体房源链接返回成功！一共有（个）： 60
https://www.ziroom.com/z/z0-p11/?qwd=%E5%8C%97%E4%BA%AC 中所有具体房源链接返回成功！一共有（个）： 60
https://www.ziroom.com/z/z0-p12/?qwd=%E5%8C%97%E4%BA%AC 中所有具体房源链接返回成功！一共有（个）： 60
https://www.ziroom.com/z/z0-p13/?qwd=

In [22]:
# 检查有效链接是多少个
len(pagesValidLinks)

1682

In [54]:
successLinks = []
failLinks = []
pagesInf = getPagesInf(pagesValidLinks, successLinks, failLinks)

已经运行到第 0 个链接，成功采集条数： 0 失败采集条数： 0
已经运行到第 100 个链接，成功采集条数： 97 失败采集条数： 3
已经运行到第 200 个链接，成功采集条数： 184 失败采集条数： 16
已经运行到第 300 个链接，成功采集条数： 284 失败采集条数： 16
已经运行到第 400 个链接，成功采集条数： 356 失败采集条数： 44
http://www.ziroom.com/x/715380417.html 打不开
已经运行到第 500 个链接，成功采集条数： 453 失败采集条数： 47
已经运行到第 600 个链接，成功采集条数： 506 失败采集条数： 94
已经运行到第 700 个链接，成功采集条数： 601 失败采集条数： 99
已经运行到第 800 个链接，成功采集条数： 701 失败采集条数： 99
http://www.ziroom.com/x/793221559.html 打不开
已经运行到第 900 个链接，成功采集条数： 794 失败采集条数： 106
已经运行到第 1000 个链接，成功采集条数： 832 失败采集条数： 168
http://www.ziroom.com/x/720757903.html 打不开
已经运行到第 1100 个链接，成功采集条数： 931 失败采集条数： 169
已经运行到第 1200 个链接，成功采集条数： 1022 失败采集条数： 178
已经运行到第 1300 个链接，成功采集条数： 1103 失败采集条数： 197
已经运行到第 1400 个链接，成功采集条数： 1125 失败采集条数： 275
已经运行到第 1500 个链接，成功采集条数： 1167 失败采集条数： 333
已经运行到第 1600 个链接，成功采集条数： 1180 失败采集条数： 420


### 处理在第一遍采集中没有成功的链接

In [136]:
# 由于上一步运行时间较长，下面使用备份变量进行操作，以方便随时回滚到此步操作
sLinks = copy.deepcopy(successLinks)
fLinks = copy.deepcopy(failLinks)

In [145]:
# 重复采集没有成功的数据
# 当重复次数 > 5 或者已经没有采集失败的链接或者连续两次采集失败的链接一模一样（网址本身可能存在问题）就停止循环
pagesInfAdd = pd.DataFrame()
for k in range(5):
    print("\n=======================正在进行第", k+1, "轮重新采集=======================")
    print("本轮开始时，成功链接条数为 {}，失败链接数为 {}.".format(len(sLinks), len(fLinks)))
    stillFailLinks = []
    # 已成功采集的链接只增不减，在下过程内部已经实现更新
    # 使用上一轮没有成功的链接作为总体，传入原本成功链接的列表以及新的依旧没有成功列表
    pagesInfAddOnce = getPagesInf(fLinks, sLinks, stillFailLinks)
    
    # 如果没有依旧没有成功的链接或者连续两次依旧没有成功的链接一模一样，结束更新
    if len(stillFailLinks) == len(fLinks) or len(stillFailLinks) == 0:
        print('【全部采集完成或者连续两次采集失败的链接完全相同】')
        break
    
    # 更新没有成功采集数据的链接
    print('***检查点：这一轮的链接总共有 {} 个，依旧失败的链接总共有 {} 个'.format(len(fLinks), len(stillFailLinks)))
    fLinks = copy.deepcopy(stillFailLinks)
    pagesInfAdd = pagesInfAdd.append(pagesInfAddOnce)
    print("***检查点：本轮增量数据框维数 {}, 补充收集过程总共增量数据框维数 {}".format(pagesInfAddOnce.shape, pagesInfAdd.shape))
    print("===========这一轮过后成功的个数", len(sLinks), "还没成功的个数为", len(fLinks), '============')
    


=======================正在进行第 1 轮重新采集=======================
本轮开始时，成功链接条数为 1224，失败链接数为 458.
已经运行到第 0 个链接，成功采集条数： 1224 失败采集条数： 0
http://www.ziroom.com/x/715380417.html 打不开
已经运行到第 100 个链接，成功采集条数： 1293 失败采集条数： 31
http://www.ziroom.com/x/793221559.html 打不开
已经运行到第 200 个链接，成功采集条数： 1392 失败采集条数： 32
http://www.ziroom.com/x/795450619.html 打不开
http://www.ziroom.com/x/715971729.html 打不开
已经运行到第 300 个链接，成功采集条数： 1489 失败采集条数： 35
http://www.ziroom.com/x/796725393.html 打不开
已经运行到第 400 个链接，成功采集条数： 1536 失败采集条数： 88
***检查点：这一轮的链接总共有 458 个，依旧失败的链接总共有 105 个
***检查点：本轮增量数据框维数 (353, 14), 补充收集过程总共增量数据框维数 (353, 14)
===========这一轮过后成功的个数 1577 还没成功的个数为 105 ============

=======================正在进行第 2 轮重新采集=======================
本轮开始时，成功链接条数为 1577，失败链接数为 105.
已经运行到第 0 个链接，成功采集条数： 1577 失败采集条数： 0
http://www.ziroom.com/x/715380417.html 打不开
http://www.ziroom.com/x/793221559.html 打不开
http://www.ziroom.com/x/795450619.html 打不开
http://www.ziroom.com/x/715971729.html 打不开
http://www.ziroom.com/x/807021574.html 打不开
http://www.

In [146]:
# 检查连续没有成功采集的链接，可舍弃
len(stillFailLinks)

6

In [147]:
# 检查最终收集到的有效数据条数
len(sLinks)

1676

In [149]:
# 备份（可省略）
# pagesInf2 = copy.deepcopy(pagesInf)

In [150]:
# 将补充收集的数据与原数据合并
pagesInf = pagesInf.append(pagesInfAdd)
pagesInf.shape

In [153]:
pagesInf.shape

(1676, 14)

## 把可以理解的价格导进去

- 获取价格的背景图片信息
- 使用 ```pytesseract``` 将图片数字转化为字符串数字
- 找到各个位数上数字对应图片的位置索引
- 生成最终价格
- 通过观察可知，每一个价格的几位数字对应的图片是同一个，对于一个价格只需要进行一次图片转字符串的操作即可

In [162]:
# 生成该数据框中的所有价格
# 返回价格列表（价格以字符串形式存在）
def getPrices(inf_df):
    
    urls = list(inf_df['url'])
    imgs = list(inf_df['img_url'])
    poses = list(inf_df['price_raw'])
    
    # 每一个循环对应一套房子的房价计算
    prices = []
    for i, pos in enumerate(poses):
        if len(prices)%100 == 0:
            print("\n已经收集了 {} 条价格信息".format(len(prices)), end = ' ')
        if len(prices)%10 == 0:
            print('-', end = '')
        ppx = pos.split(",")
        # 一个价格映射到的图片文件相同，识别一次即可
        BytesIOObj = BytesIO()
        response = requests.get(imgs[i])
        response = response.content
        BytesIOObj.write(response)
        img = Image.open(BytesIOObj)
        # 将图片转化为字符串
        imgText = pytesseract.image_to_string(img).replace(" ", "")
        # 识别各个位数的数字
        price = ''
        for p in ppx:
            pIdx = int(float(p)/30.4)
            num = imgText[pIdx]
            price += num
        prices.append(price)
    
    return prices

In [155]:
# 将价格加在数据框上
def setPrices(inf_df, prices_in):
    inf_df['price'] = prices_in

In [163]:
prices = getPrices(pagesInf)


已经收集了 0 条价格信息 ----------
已经收集了 100 条价格信息 ----------
已经收集了 200 条价格信息 ----------
已经收集了 300 条价格信息 ----------
已经收集了 400 条价格信息 ----------
已经收集了 500 条价格信息 ----------
已经收集了 600 条价格信息 ----------
已经收集了 700 条价格信息 ----------
已经收集了 800 条价格信息 ----------
已经收集了 900 条价格信息 ----------
已经收集了 1000 条价格信息 ----------
已经收集了 1100 条价格信息 ----------
已经收集了 1200 条价格信息 ----------
已经收集了 1300 条价格信息 ----------
已经收集了 1400 条价格信息 ----------
已经收集了 1500 条价格信息 ----------
已经收集了 1600 条价格信息 --------

In [173]:
# 将价格的类型转化为整数型
price_nums = [int(price) for price in prices]

In [177]:
# 将价格导入数据框
setPrices(pagesInf, price_nums)

In [192]:
# 存储最终结果
pagesInf.to_csv(index = False, header = True, path_or_buf="C:/Users/lenovo/Desktop/zirudata.csv", encoding = "utf_8_sig",
               date_format = '-')